### This is a jupyter notebook that predicts the next character based on previous characters of a word. It can also intelligently predict if we know the character we are going to predict is the last character of any given word.

### We create a trie of all words in document which is effectively a nested dictionary, search through the tries using given letters of a word and predict the next letter based on maximum frequency of the letter

In [1]:
#FILE = "../data/words.txt"                  # ~90%
#FILE = "../data/hp_sorcerers_stone.txt"     # 80%
#FILE = "../data/pride_and_prejudice.txt"    # 83.9%
FILE = "../data/ptb.train.txt"              # >86%

In [2]:
# Built and tested on python2
import numpy as np
from random import randint
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Functions for working with tries and generating predictions:

In [3]:
_first = '!@#$%^&*()'      # String to decide the first/last character of word in trie
_end   = '!@#$%^&*()'
_count = 'count'

# Takes in list of words and returns a trie which is a nested dictionary of lowercase letters
def make_trie(words):
    root = dict()
    for word in tqdm(words):
        current_dict = root
        current_dict[_first] = 'first'
        current_dict[_count] = current_dict.setdefault(_count, 0) + 1
        
        for letter in word.lower():
            # in dict.setdefault:
            #   if (key, value) pair present: returns the value (which is dictionary)
            #   else: sets dict[key]=default and returns the value (i.e default) (which is dictionary)
            # making current_dict point to value of key of dictionary
            current_dict = current_dict.setdefault(letter, {})
            current_dict[_count] = current_dict.setdefault(_count, 0) + 1
        current_dict[_end] = 'last'
    return root
 
# Takes in trie and word and says if the word is present or not
def in_trie(trie, word):
    current_dict = trie
    for letter in word.lower():
        if letter not in current_dict:
            return False
        current_dict = current_dict[letter]
    if _end not in current_dict:
        return False
    return True

# Takes in trie and word and inserts the word in trie if not present
def insert_word(trie, word):
    current_dict = trie
    current_dict[_count] = current_dict.setdefault(_count, 0) + 1
    for letter in word.lower():
        current_dict = current_dict.setdefault(letter, {})
        current_dict[_count] = current_dict.setdefault(_count, 0) + 1
    current_dict[_end] = 'last'
    
# Takes in trie and word and gives its count in dictionary
def get_count(trie, word):
    current_dict = trie
    for letter in word.lower():
        if letter not in current_dict:
            return 0
        current_dict = current_dict[letter]
    return current_dict[_count]

# For a given word, predicts [next character, confidence] as per nested dictionary
# If last_character is set to True, it returns a letter only if next letter is last letter of any word in dictionary
def predict_next_letter(trie, word, last_character):
    current_dict = trie
    
    # Iterate over given letters
    for letter in word.lower():
        if letter not in current_dict:
            return [None, None]
        current_dict = current_dict[letter]
    
    # Determine count for all child dictionaries
    max_count   = 0
    total_count = 0
    prediction_list = []
    for letter, child_dict in current_dict.iteritems():
        if type(child_dict) is dict:
            if last_character and _end not in child_dict:
                continue
            
            total_count = total_count + child_dict[_count]
            if max_count < child_dict[_count]:
                max_count = child_dict[_count]
                prediction_list = [letter]
            elif max_count == child_dict[_count]:
                prediction_list.append(letter)
    
    if len(prediction_list) == 0:    # If no letter found, check if it is the last letter of a word
        if not last_character and _end in current_dict:
            result = [' ', 1.0]
        else:
            result = [None, None]
    elif len(prediction_list) == 1:
        result = [prediction_list[0], (1.0 * max_count / total_count)]
    else:                            # If more than 1 letter have same maxm frequency, choose any one letter at random
        idx = randint(0, len(prediction_list)-1)
        result = [prediction_list[idx], (1.0 * max_count / total_count)]

    return result

# Evaluate Predictions
def evaluate_predictions(trie, x_test_list, y_test_list, last_character, guess, log):
    correct_prediction = 0;
    prediction_list = []
    confidence_list = []
    
    for i in tqdm(range(len(x_test_list))):
        word = x_test_list[i]
        [predicted_letter, confidence] = predict_next_letter(trie, word, last_character=last_character)
        
        # Normally we should leave it alone, but traditional ML algos will make a guess
        if guess and predicted_letter is None:
            idx = randint(0, 25)
            predicted_letter = chr(ord('a') + idx)
        
        if predicted_letter is not None and predicted_letter == y_test_list[i].lower():
            correct_prediction = correct_prediction + 1
        
        if log:
            prediction_list.append(predicted_letter)
            confidence_list.append(confidence)
    
    accuracy = correct_prediction * 1.0 / len(x_test_list)
    return [accuracy, prediction_list, confidence_list]

Functions for file I/O and data handling:

In [4]:
# Take filename as input and returns np list of words
def read_text_to_words(file):
    words_list = []
    with open(file) as f:
        for line in tqdm(f):
            for word in line.split():
                words_list.append(word)
    return words_list

# Take word_list and return train and test set like scikit
def get_train_test_set(word_list, test_size, random_state):
    X = [word[:-1] for word in word_list]
    y = [word[-1] for word in word_list]
    if random_state is None:
        return train_test_split(X, y, test_size=test_size)
    else:
        return train_test_split(X, y, test_size=test_size, random_state=random_state)

# Rejoining training data and labels to feed to the trie
def prepare_train_data(x_train):
    train = x_train[:]
    for t, y in zip(train, y_train):
        yield ''.join([t, y])



##### Test script starts here:

In [5]:
WORD_LIST = read_text_to_words(FILE)

42068it [00:00, 94980.68it/s]


In [6]:
# you can change test_size and random_state here

[X_train, X_test, y_train, y_test] = get_train_test_set(WORD_LIST, test_size=0.25, random_state=None)

print('Generate train-test split: done')

Generate train-test split: done


In [7]:
train_data = list(prepare_train_data(X_train))

print('Prepare train data for trie: done')

Prepare train data for trie: done


In [8]:
# Build the trie
TRIE = make_trie(WORD_LIST)

100%|██████████| 887521/887521 [00:05<00:00, 158546.65it/s]


In [9]:
# last_character: set it True if you want to only predict last character of word and make use of this fact
# guess: set it True if you want random character in case of no predictions
# log: set it True if you want to see individual predictions and confidence score; not memory-friendly

train_accuracy = evaluate_predictions(TRIE, X_test, y_test, last_character=False, guess=True, log=False)
test_accuracy = evaluate_predictions(TRIE, X_test, y_test, last_character=False, guess=True, log=False)

print("Train accuracy w/o last_character knowledge: {}%".format(train_accuracy[0]*100))
print("Test accuracy w/o last_character knowledge: {}%".format( test_accuracy[0]*100))

100%|██████████| 221881/221881 [00:03<00:00, 60868.93it/s]

Train accuracy w/o last_character knowledge: 76.5293107567%
Test accuracy w/o last_character knowledge: 76.5284093726%


In [10]:
train_accuracy = evaluate_predictions(TRIE, X_train, y_train, last_character=True, guess=True, log=False)
test_accuracy = evaluate_predictions(TRIE, X_test, y_test, last_character=True, guess=True, log=False)

print("Train accuracy with last_character knowledge: {}%".format(train_accuracy[0]*100))
print("Test accuracy with last_character knowledge: {}%".format( test_accuracy[0]*100))

100%|██████████| 221881/221881 [00:03<00:00, 66489.23it/s]

Train accuracy with last_character knowledge: 86.2174749114%
Test accuracy with last_character knowledge: 86.3390736476%


The above difference between last character knowledge shows that we can make intelligent predictions if we know the character we are going to predict is the last character of the word. In such case, we willingly look for word-end-marker and only include those letters for predictions, weeding out other irrelevant letters that wont end up forming a word.

This is cool here but in general, we may not know if the letter we are going to predict is last or not, so keeping it false should be the way to go forward.